# 데이터 추출 및 훈련 데이터 생성

In [1]:
import pandas as pd

# NSMC 훈련용 코퍼스 내 문장 및 라벨 데이터 각각 추출
f_train = pd.read_csv('ratings_train.txt', sep='\t')
train_pair = [(row[1], row[2]) for _, row in f_train.iterrows() if type(row[1]) == str]

train_data = [pair[0] for pair in train_pair]
train_label = [pair[1] for pair in train_pair]

In [3]:
# 추출된 문장 데이터 일부 확인
for data, label in zip(train_data[:3], train_label[:3]):
    print(f'Data: {data} |\t Label : {label}')

Data: 아 더빙.. 진짜 짜증나네요 목소리 |	 Label : 0
Data: 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 |	 Label : 1
Data: 너무재밓었다그래서보는것을추천한다 |	 Label : 0


# Tokenizers 라이브러리 활용한 토크나이저 학습

In [4]:
# 추출한 문장 데이터 텍스트 파일로 저장
with open('train_tokenizer.txt', 'w', encoding='utf-8') as f:
    for line in train_data:
        print(line, file=f)

In [16]:
from tokenizers import BPETokenizer

# 토크나이저 초기화
tokenizer = BPETokenizer()

# 앞서 제작한 텍스트 파일 활용해 토크나이저 훈련
tokenizer.train(
    'train_tokenizer.txt',
    vocab_size=20000,
    min_frequency=3,
    suffix='',
    special_tokens=['<PAD>', '<SOS>', '<EOS>', '<UNK>'],
)

In [52]:
# 패딩 인덱스 확인
pad_idx = tokenizer.token_to_id('<PAD>')
pad_idx

2

In [129]:
eos_idx = tokenizer.token_to_id('<EOS>')

In [18]:
# 토크나이저에 대해 패딩 옵션 설정
tokenizer.enable_padding(pad_id=pad_idx, pad_token='<PAD>', max_length=32)

In [61]:
# 토크나이저 훈련 결과 확인
encoded = tokenizer.encode('너무 재밌게 봤어요 스토리 예상가나 싶었는데 그래도 놀라는 반전이있는 ㅎㅎ')
print(encoded.ids)
print(encoded.tokens)

[1011, 1160, 1468, 1036, 1977, 9157, 6605, 1308, 13305, 2510, 1046, 1091, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
['너무', '재밌게', '봤어요', '스토리', '예상', '가나', '싶었는데', '그래도', '놀라는', '반전이', '있는', 'ᄒᄒ', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [340]:
# encode_batch 함수 이용해 훈련 데이터셋에 대해 토크나이즈 수행
encoded_data = tokenizer.encode_batch(train_data)

In [330]:
print(f'훈련 데이터: {len(train_data)} 개')
print(f'훈련 라벨: {len(train_label)} 개')
print(f'인코딩 데이터: {len(encoded_data)} 개')

훈련 데이터: 149995 개
훈련 라벨: 149995 개
인코딩 데이터: 149995 개


In [331]:
print(f'토큰: {encoded_data[2020].tokens}\n')
print(f'아이디: {encoded_data[2020].ids}')

토큰: ['20', '여', '년이', '흘러', '두', '기억에', '남는', '사랑에', '대해', '생각해', '볼수있는', '유쾌한', '영화', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

아이디: [1216, 647, 1958, 2402, 291, 1665, 1310, 2877, 1827, 2162, 3922, 2863, 1005, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [43]:
# 단어 사전 구축
vocab = {}

for idx in range(20000):
    word = tokenizer.id_to_token(idx)
    id = tokenizer.token_to_id(word)
    vocab[word] = id

In [332]:
# 후처리 함수 정의
def postprocess(input_ids):
    input_ids = [vocab['<SOS>']] + input_ids
    input_ids = input_ids[:32]

    if pad_idx in input_ids:
        pad_start = input_ids.index(pad_idx)
        input_ids[pad_start] = vocab['<EOS>']
    else:
        input_ids[-1] = vocab['<EOS>']
    
    return input_ids

In [342]:
processed_data = [postprocess(data.ids) for data in encoded_data]

In [334]:
print(f'후처리 결과: {processed_data[0]}\n')
print(f'후처리 결과 디코딩: {tokenizer.decode(processed_data[0])}')

후처리 결과: [1, 612, 1623, 1004, 1022, 12821, 2004, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

후처리 결과 디코딩: <SOS>아더빙..진짜짜증나네요목소리<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>


# 간단한 Bi-LSTM 분류기 구현

In [336]:
import torch
import torch.nn as nn
from torch.utils import data
from torch.nn.utils.rnn import pack_padded_sequence

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [350]:
class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(20000, 200, padding_idx=pad_idx)
        self.lstm = nn.LSTM(
            input_size=200,
            hidden_size=256,
            num_layers=2,
            bidirectional=True,
            dropout=0.1
        )
        self.fc = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, input_ids, input_lengths):
        embedded = self.embedding(input_ids)
        print(input_lengths)
        input_lengths = input_lengths.view(-1, 16)
        
        packed_embedded = pack_padded_sequence(embedded, input_lengths)
        
        _, (hidden, cell) = self.lstm(packed_embedded)
        
        
        cat_hidden = torch.cat((hidden[-1], hidden[-2]), dim=-1)
        output = self.dropout(cat_hidden)
        return self.fc(cat_hidden)

In [343]:
input_lengths = [data.index(eos_idx)+1 for data in processed_data]

processed_data = [torch.LongTensor(data).to(device) for data in processed_data]
input_lengths = [torch.LongTensor([length]).to(device) for length in input_lengths]

train_iter = data.DataLoader(processed_data, batch_size=16)
lengths_iter = data.DataLoader(input_lengths, batch_size=16)

In [351]:
model = BiLSTM()
model.to(device)

for (batch, lengths) in zip(train_iter, lengths_iter):
    model(batch, lengths)
    break

tensor([[ 8],
        [15],
        [ 9],
        [12],
        [26],
        [20],
        [ 9],
        [32],
        [ 9],
        [18],
        [10],
        [20],
        [17],
        [24],
        [22],
        [ 9]], device='cuda:0')


RuntimeError: 'lengths' argument should be a 1D CPU int64 tensor